# ETL with Python - Manipulating Large Masses of Data

_Este arquivo é o resultado do projeto guiado instruído pela DIO realizado em junho de 2024._

## **PROBLEMA**

Uma organização recebe um conjunto de dados de um relatório de forma constante e é esperado que eles sejam organizados em um painel que sintetize todos eles de uma maneira clara. Esses dados de assinaturas são gerados diariamente e representam os produtos gerados por cada sistema distribuído em cada país. Como consequência, eles não possuem a mesma padronização.

Ao final será entregue uma aplicação que faz a carga de arquivos XLSX em Bancos de Dados.

Este estudo mostra **maturidade do processo** e **a visibilidade que se tem sobre os dados**.




## CONTEÚDO APRENDIDO

* **Organização de um projeto de Tratamentos de Dados**
    * Organização de as pastas
    * Criação de ambiente de execução (venv)

* Etapas de dados
* Conectar nos Bancos de Dados
* Garantir a confiabilidade dos dados
* Garantir a rastreabilidade dos dados
* Desenhar esquemas (tldraw - extensão no VSCode)

**CONCEITOS PARA ENTREVISTA**

Para demonstrar MATURIDADE e CONHECIMENTO DO SENTIDO das pequenas partes.

Muitas vezes, em entrevistas técnicas é comum a pesso achegar com ocódigo pronto. Muitas vezes é interessante explicar os principais conceitos (porque usar uma determinada solução? Como ela funciona? Para uso de VENV, organização do projeto e camadas de dados). **Demonstrar que sabe fazer o simples bem feito**.

Então saber falar sobre o desenvolvimento de SOLUÇÕES e não somente de CÓDIGOS.

**Regras do Tratamento de dados**

* Prezar pela confiabilidade e rastreabilidade dos dados

**Camadas de Armazenamento**

* RAW e READY
* BRONZE, SILVER e GOLD

**Ambiente Virtual Execução/Desenvolvimento**

**ETL de Dados**

* cookiecutter - módulo de python



**UTM tag**

UTM (Urchin Tracking Module ou Módulo de Rastreamento Urchin) é uma extensão de link de rastreamento que serve para entender a origem do tráfego.

Através da UTM, é possível saber onde foi publicado e qual o conteúdo e campanha veiculados a determinado link

### Estrutura de Projeto de Tratamento de Dados

* `src`
    * _Tudo dentro do source é o código-fonte ou arquivos de textos, imagens etc. Tudo que estiver fora da `src` ou é documentação ou é configuração_
    * `data`
        * `raw`: dados brutos sem qualquer alteração
        * `ready`: dados que já passaram por alguns script de tratamento do projeto. Então é onde os dados refinados são armazenados.
    * `main.py`: se for só um arquivo ou uma aplicação inicial que chama todas as outras é possível usar um único arquivo.
    * [`scripts`]: pasta opcional para quando hpa mais do que um único arquivo de execução da aplicação. Isso serve para separar a camada de dados da camada de códigos.


Uma estrutura de projeto de API não é igual a uma estrutura de projeto de dados

```
data/
|- raw/
|- ready/
|- main.log
scripts/
|- notebooks/
|- main.py
docs/
|- how-to.md
|- changelog.md
```

"Transformação de data raw em dados refinados de negócios"

### Padrão Arquitetural - MVC

https://www.youtube.com/watch?v=9Ieh0yoiiqI

Um padrão arquitetural é uma forma de estruturar uma aplicação, como um todo, com todas as suas funcionalidades. Um padrão é um desenho de camadas, principalmente, camadas lógicas.

Um padrão arquitetural é importante é importante para planejar um Framework. Desta forma, ela ajuda a separar as partes da solução em camadas lógicas (pastas), contribuindo para a organização e independência entre estes grandes elementos da arquitetura. Isso segue o princípio do **desacoplamento**. Uma vantagem é a simplificação do processo de testes, em que pode-se testar cada um desses elementos individualmente. Então melhora a **TESTABILIDADE**.

* CONTROLLER é a camada principal, também chamada de camada primária. Então ela é a porta de entrada para qualquer cliente. Por exemplo em um site, o cliente acessa através de uma rota o `controller` que irá carregar uma `view`. Esta camada possuirá códigos de orquestração de chamada de `models` quanto de `views`.

* **VIEW**: esta camada é responsável por trabalhar todas as regras de visualização, ou seja, aspectos relacionados com a experiência com o usuário (regras de input e output; interface com o usuário).

* **MODEL**: esta camada possui todas as lógicas da regra de negócio.


Para fazer este direcionamento, a `controller`possuirá uma lógica/inteligência que irá carregar informações de um `model`, que será processadas para construir uma `view`. A resposta da `view` é enviada para o `Controller`, que enviará para o usuário.

### Git Flow

1. Nova Feature
    1. AMBIETE DE DESENVOLVIMENTO: Criar nova Branch LOCAL `<ftr|bug|done>/<..>`
    2. Desenvolver a solução
    3. UPDATE: Fazer o pull para o `main`
    3. MERGE: Merge into main LOCAL
    4. PULL REQUEST: Fazer o push para uma nova Branch, a partir do `main local` a uma nova Branch Remota. Depois fazer o Pull Request desta Branch

## TAREFAS

* Consolidar diversos arquivos de planilhas em uma única fonte
    * Extrair informações de strings
    * Rastreabildiade dos dados - indicar qual a origem dos dados

## ROTEIRO DE PROJETO

1. <font color=orange><b>Iniciar um Novo Projeto</b></font>
    1. Abrir pasta
        * `C:/www/python`
    2. Nova pasta para o Projeto
        * `mkdir make-data-netflix`
    3. Abrir o VSCode na pasta
        * `code make-data-netflix`
2. <font color=orange><b>Documentação do Projeto</b></font>
    1. Crição de Pasta
        * `mkdir docs`
        * `cd docs`
        * `cat how-to.md`
3. <font color=orange><b>Ambiente de Desenvolvimento</b></font>
    1. Criação do ambiente
        * `python -m venv .venv`
    2. Ativação do ambiente
        * `.venv/Scripts/activate`
    3. Instalação de dependências
        * `pip install pandas, openpyxl, xlsxwriter, requests, polars`
        * `xlswriter` é mais performático na hora de escrever e permite operar de maneira mais atômica.
        * `pip install -r requirements.txt`
        * `poetry`
    4. Git
        * `.gitignore`
            * adicionar a pasta `src/data/`
            * adicionar a pasta `.venv/`
3. <font color=orange><b>Preparação</b></font>
    1.

1. Adicionar no Github
2. Adicionar Github do projeto no <u>Repositório de Aulas</u>
3. Criar

* [Repositório de dados](https://www.youtube.com/redirect?event=live_chat&redir_token=QUFFLUhqbjFNdkpmNGhMTzROX2lJWGhXMUVYOW0xZWFtUXxBQ3Jtc0ttbUphNDV1dklmUEE4VjlnTWpONTVGSV8zeHZ5UG9CVWlHRWlfM1lvc3B5OV9FXzVkT2lyVk9LYWZqX3d3OVJVRWhZUDNNVFEtVWpObWxpWXlockhNbDN5WmZWUFBSLXdYRkJIaUduZFJCTE9VUTZ0RQ&q=https%3A%2F%2Fgithub.com%2Fdigitalinnovationone%2Fnetflix-dataset%2Ftree%2Fmain%2Fraw)

In [ ]:
import pandas as pd
import os
import glob

# Caminho para ler o caminho
folder_path = os.path.join(os.getcwd(), 'src', 'data', 'raw')

# Lista de arquivos
glob_unix_pattern = os.path.join(folder_path, '*.xlsx')
excel_files = glob.glob( glob_unix_pattern )

if not excel_files:
    print('No file found in the specified folder')
    raise ValueError('No file found in the specified folder')
else:
    # Dataframe - data in memory
    dfs = []

    for excel_file in excel_files:
        try:
            # Leitura de XLSX
            df_temp = pd.read_excel(excel_file)

            # Principio da rastreabilidade
            # Nome do arquivo
            df_temp['filename'] = os.path.basename(excel_file)

            # Extração da localização da tabela
            df_temp['location'] = df_temp['filename'].str.split('_')[-1]
            df_temp['location'] = df_temp['location'].str.split('.')[0].str.lower()

            # Extração de info sobre campaign - UTM
            # df_temp = df_temp['utm_link'].str.split('utm_source=').str[-1]
            df_temp = df_temp['utm_link'].str.extract(r'utm_campaign=(.*)')  # no extract recebe um padrão de regex

            df_temp['campaign'] = df_temp[df_temp['utm_source'].notnull()]['utm_campaign']
            df_temp['utm_source'] = df_temp[df_temp['utm_source'].notnull()]['utm_source']

            # Conjunto de tabelas
            dfs.append(df_temp)

        except Exception as e:
            print(f'Erro ao ler o arquivo {excel_file}: {e}')


if len(dfs) != 0:
    # Concatena todas as tabelas contidas no dfs em um única tabela
    result = pd.concat( dfs, ignore_index=True)

    # Cria o caminho da nova tabela tratada
    output_file = os.path.join(os.getcwd(), 'src', 'data', 'ready', 'netflix_data_cleaned.xlsx')

    # Criação da instância com especificação de motor (opcional)
    writer = pd.ExcelWriter(output_file, engine='xlsxwriter')  # mudança do motor de escrita. XLSXWRITER pode ser mais performático

    # Persistência da tabela em um arquivo excel e fechamento do arquivo
    result.to_excel(writer, sheet_name='netflix_data_cleaned', index=False)
    writer.save()
    writer.close()
else:
    print('Nenhum dado para ser salvo')

# BOAS PRÁTICAS

```python
```

* Código robusto entre sistemas

```python
# Caminho para ler o caminho
folder_path = os.path.join(os.getcwd(), 'src', 'data', 'raw')
```

* Listagem de arquivos com filtro

```python
# Lista de arquivos
glob_unix_pattern = os.path.join(folder_path, '*.xlsx')
excel_files = glob.glob( glob_unix_pattern )
```

* Design por Contrato

```python
if not excel_files:
    print('No file found in the specified folder')
    raise ValueError('No file found in the specified folder')
```

* Tolerante a falhas

```python
except Exception as e:
    print(f'Erro ao ler o arquivo {excel_file}: {e}')
```

* Operações Pandas

```python
# Extração da localização da tabela
df_temp['location'] = df_temp['filename'].str.split('_')[-1]
df_temp['location'] = df_temp['location'].str.split('.')[0].str.lower()

# Extração de info sobre campaign - UTM
# df_temp = df_temp['utm_link'].str.split('utm_source=').str[-1]
df_temp = df_temp['utm_link'].str.extract(r'utm_campaign=(.*)')  # no extract recebe um padrão de regex
```



```python
```

# README



1. Definição dos Requisitos de Entrada e Saída
2. Definição dos Padrões e das Transformações necessárias
    * Definição da Organização do Projeto
    * Definição das Camadas de Especilização dos Dados
    * Virtualização do ambiente de desenvolvimento
1. Definição das sprints
    * Processo incremental de melhorias
3. Construção do primeiro protótipo

<font color=orange>SPRINTS TEMÁTICAS</font>

* Modulação da solução
* Controle do fluxo, resiliência e contratos da solução
    * Gerenciamento de erros
* Qualidade de Dados
* Rastreabilidade dos dados
    * Nomes dos arquivos
    * Link UTM
* Performance das soluções e alternativas
* Encapsulamento da solução
* Conteinerização
* Orquestração
* Sistema de Logs

**ROTEIRO DE DESENVOLVIMENTO DE SOLUÇÃO DE ENGENHARIA DE DADOS**

1. PLANEJAMENTO DO PROBLEMA

    * A. **Entendimento do Problema**: _Definição clara do problema que está tentando resolver com a solução de dados._
    * B. **Entendimento dos Requisitos de Negócio (Objetivos)**: _Definição dos objetivos da solução de dados._
        * Quais problemas de negócio precisa ser resolvido?
    * C. **Definição dos Requisitos de Dados**: _Definição de parâmetros esperados de desempenho_.
        * Qual o volume, a variedade e a velocidade dos dados que serão processados?
        * Quais as fontes de dados?
        * Qual a frequencia de atualização?
    * D. **Arquitetura da Solução**: Escolha das tecnologias e ferramentas adequadas para cada etapa do pipeline de dados, considerando aspectos como escalabilidade, performance, custo e segurança.
        * **Armazenamento de Dados**: Data Lakes (S3, Azure Glob Storage, MinIO); Data Warehouse (Redshift, BigQuery), Bancos de Dados NoSQL (Cassandra, MongoDB, ScyllaDB)
        * **Processamento de Dados**: Ferramentas de ETL (Apache Spark, Apache Beam), plataformas de streaming / mensageria (Kafka, Kinesis)
        * **Orquestração de Pipelines**: Ferramentas como Apache Airflow ou Luigi.
    
2. COLETA E INGESTÃO DE DADOS

    * Construa Pipelines de Ingestão: Crie pipelines robustos e escaláveis para coletar dados de diversas fontes (bancos de dados, APIs, arquivos, sensores, etc.).
    * Garanta a Qualidade dos Dados: Implemente mecanismos de validação e limpeza de dados durante a ingestão para garantir a consistência e a confiabilidade dos dados.

3. ARMAZENAMENTO E PROCESSAMENTO

*Modelagem de Dados: Defina a estrutura dos dados para armazenamento e processamento, considerando os requisitos de análise e as ferramentas a serem utilizadas.
Processamento em Batch ou Streaming: Escolha a abordagem de processamento mais adequada (batch ou streaming) com base na frequência de atualização dos dados e nos requisitos de tempo real.
Otimização de Performance: Utilize técnicas de otimização para garantir o processamento eficiente de grandes volumes de dados.



Um roteiro para uma solução de engenharia de dados pode ser mais específico, focando em aspectos como infraestrutura, processamento de dados em larga escala e pipelines de dados. Aqui está um roteiro detalhado:

1. Definição de Requisitos e Arquitetura:

Entenda os Requisitos de Negócio: Defina os objetivos da solução de dados. Quais problemas de negócio ela precisa resolver? Quais insights ela precisa fornecer?
Defina os Requisitos de Dados: Determine o volume, a variedade e a velocidade dos dados que serão processados. Quais as fontes de dados? Qual a frequência de atualização?
Desenhe a Arquitetura da Solução: Escolha as tecnologias e ferramentas adequadas para cada etapa do pipeline de dados, considerando aspectos como escalabilidade, performance, custo e segurança. Isso pode incluir:
Armazenamento de Dados: Data lakes (S3, Azure Data Lake), data warehouses (Redshift, BigQuery), bancos de dados NoSQL (Cassandra, MongoDB).
Processamento de Dados: Ferramentas de ETL (Apache Spark, Apache Beam), plataformas de streaming (Kafka, Kinesis).
Orquestração de Pipelines: Ferramentas como Apache Airflow ou Luigi.
2. Coleta e Ingestão de Dados:

Construa Pipelines de Ingestão: Crie pipelines robustos e escaláveis para coletar dados de diversas fontes (bancos de dados, APIs, arquivos, sensores, etc.).
Garanta a Qualidade dos Dados: Implemente mecanismos de validação e limpeza de dados durante a ingestão para garantir a consistência e a confiabilidade dos dados.
3. Armazenamento e Processamento de Dados:

Modelagem de Dados: Defina a estrutura dos dados para armazenamento e processamento, considerando os requisitos de análise e as ferramentas a serem utilizadas.
Processamento em Batch ou Streaming: Escolha a abordagem de processamento mais adequada (batch ou streaming) com base na frequência de atualização dos dados e nos requisitos de tempo real.
Otimização de Performance: Utilize técnicas de otimização para garantir o processamento eficiente de grandes volumes de dados.
4. Governança e Segurança de Dados:

Implemente Políticas de Governança: Defina políticas de acesso, controle de qualidade, privacidade e segurança dos dados.
Garanta a Segurança dos Dados: Utilize mecanismos de autenticação, autorização e criptografia para proteger os dados em todas as etapas do pipeline.
5. Monitoramento e Operação da Solução:

Implemente Monitoramento Contínuo: Monitore a saúde e a performance do pipeline de dados, coletando métricas e logs para identificar e solucionar problemas rapidamente.
Automatize Tarefas Operacionais: Utilize ferramentas de automação para simplificar tarefas como provisionamento de recursos, implantação de código e gerenciamento de configuração.
6. Documentação e Comunicação:

Documente a Solução: Crie documentação detalhada sobre a arquitetura, os pipelines de dados, as ferramentas utilizadas e os processos operacionais.
Comunique os Resultados: Mantenha as partes interessadas informadas sobre o progresso do projeto, os desafios encontrados e os resultados alcançados.
Este roteiro fornece uma visão geral das etapas envolvidas na construção de uma solução de engenharia de dados. A implementação específica de cada etapa dependerá dos requisitos do projeto, das tecnologias escolhidas e das habilidades da equipe.

**ROTEIRO DE DESENVOLVIMENTO DE SOLUÇÃO DE DADOS**

1. PLANEJAMENTO DO PROBLEMA

    * A. **Entendimento do Problema**: _Definição clara do problema que está tentando resolver com a solução de dados.
        * Quais perguntas busca responder?
        * Quais insights deseja obter?
    * B. **Definição dos Objetivos**: _Estabelecimento de objetivos mensuráveis e específicos para o projeto_.
        * O que espera alcançar com o projeto?

Aqui está um roteiro de checklist para as etapas de desenvolvimento de uma solução de dados, baseado no script que você forneceu:

1. Planejamento e Definição do Problema:

Entendimento do Problema: Defina claramente o problema que você está tentando resolver com a solução de dados. Quais perguntas você busca responder? Quais insights você deseja obter?
Definição dos Objetivos: Estabeleça objetivos mensuráveis e específicos para o projeto. O que você espera alcançar com a análise dos dados?
Identificação das Fontes de Dados: Determine quais fontes de dados serão utilizadas e como os dados serão coletados.
Definição da Arquitetura da Solução: Planeje a estrutura da solução, incluindo as ferramentas e tecnologias a serem utilizadas (bancos de dados, plataformas de processamento, ferramentas de visualização, etc.).
2. Coleta e Armazenamento de Dados:

Extração dos Dados: Utilize bibliotecas como Pandas para extrair os dados das fontes identificadas (arquivos XLSX, APIs, bancos de dados, etc.).
Limpeza e Pré-processamento dos Dados: Realize a limpeza dos dados, tratando valores ausentes, inconsistências e erros.
Armazenamento dos Dados: Armazene os dados em um formato adequado para análise, como um banco de dados relacional (MySQL, PostgreSQL) ou um data lake (Hadoop, S3).
3. Processamento e Transformação de Dados:

Transformação dos Dados: Utilize ferramentas de ETL (Extract, Transform, Load) para transformar os dados em um formato adequado para análise. Isso pode incluir agregação, filtragem, junção de tabelas, etc.
Criação de Features: Engenharia de recursos para criar novas variáveis a partir dos dados existentes, que podem melhorar a performance dos modelos de machine learning.
Normalização e Padronização dos Dados: Aplique técnicas de normalização e padronização para garantir que os dados estejam em uma escala adequada para análise.
4. Análise e Modelagem de Dados:

Análise Exploratória de Dados: Realize uma análise exploratória dos dados para entender as distribuições, identificar padrões e gerar insights.
Modelagem de Dados: Utilize técnicas de machine learning para construir modelos preditivos ou descritivos, dependendo dos objetivos do projeto.
Validação dos Modelos: Avalie a performance dos modelos utilizando métricas apropriadas e técnicas de validação cruzada.
5. Visualização e Comunicação dos Resultados:

Criação de Visualizações: Utilize ferramentas de visualização (Matplotlib, Seaborn, Plotly) para criar gráficos e dashboards que comuniquem os insights de forma clara e eficaz.
Interpretação dos Resultados: Interprete os resultados da análise e os insights gerados pelos modelos, traduzindo-os em linguagem de negócios.
Comunicação dos Resultados: Apresente os resultados e insights de forma clara e concisa para a audiência, utilizando relatórios, apresentações ou dashboards interativos.
6. Implantação e Monitoramento da Solução:

Implantação da Solução: Implemente a solução em um ambiente de produção, garantindo que ela seja escalável e robusta.
Monitoramento da Solução: Monitore a performance da solução e os resultados dos modelos, realizando ajustes e atualizações quando necessário.
7. Documentação:

Documente todo o processo: Desde a definição do problema até a implantação da solução, documentando as decisões tomadas, os códigos desenvolvidos e os resultados obtidos.
Este roteiro serve como um guia geral e pode ser adaptado de acordo com as necessidades específicas do seu projeto. Lembre-se de que a comunicação e a colaboração entre as equipes são essenciais para o sucesso de qualquer projeto de dados.

# ETL Pipeline XLSX

Este projeto consiste em um processo de ETL simples para consolidar dados de múltiplos arquivos XLSX em um único arquivo.

O código Python utiliza bibliotecas como Pandas, Openpyxl, Xlsxwriter para manipular os dados localizados em um repositório local

## Funcionalidades

* Extração de dados de vários arquivos XLSX em um pasta específica (`src/data/raw`)
* Adição de informações de rastreabilidade, como nome do arquivo, localidade e informações de campanha a partir de links UTM
* Concatenação dos dados em um único DataFrame
* Persistência do DataFrame em um novo arquivo XLSX


## Como usar

* <font color=red>Conteinerizar solução</font>

1. Preparação do ambiente de execução

`pip install -r requirements.txt`

2. Os arquivos XLSX devem estar na pasta `src/data/raw`.

3. Execute o script python

4. O arquivo consolidado será salvo em `src/data/ready/data_cleaned.xlsx`


## Conclusões

Este projeto serve como base para construir pipelines de dados mais complexos e pode ser adaptado para diferentes fontes de dados e requisitos de transformação.